In [156]:
import numpy as np
import pandas as pd
import networkx as nx



import os
import sys
import inspect
# currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
# parentdir = os.path.dirname(currentdir)
# sys.path.insert(0, parentdir)

sys.path.insert(0, '..')

import Utils as Utils

import scanpy as sc

import mygene

In [158]:
data_path = "scRNA-Seq-Spatial Multiomics Reveals Metabolic Reprogramming and Calcification Characteristics of Diabetic Macroangiopathy/";
file_name = "GSE248608_RAW";

adata = sc.read_10x_mtx(data_path+file_name);
adata


AnnData object with n_obs × n_vars = 4992 × 37487
    var: 'gene_ids', 'feature_types'

In [159]:
adata.var["mito"] = adata.var_names.str.startswith("MT-");
sc.pp.filter_cells(adata, min_genes=2000);
sc.pp.filter_genes(adata, min_cells=700);
adata

AnnData object with n_obs × n_vars = 961 × 630
    obs: 'n_genes'
    var: 'gene_ids', 'feature_types', 'mito', 'n_cells'

In [162]:
# Normalizing to median total counts
sc.pp.normalize_total(adata)
# Logarithmize the data
sc.pp.log1p(adata)


In [164]:
sc.pp.highly_variable_genes(adata, n_top_genes=300, inplace=True);
adata


AnnData object with n_obs × n_vars = 961 × 630
    obs: 'n_genes'
    var: 'gene_ids', 'feature_types', 'mito', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg'

In [166]:
adata.var.highly_variable
adata.X
print(adata)
print(adata.X)
display(adata.obs)
display(adata.var)

AnnData object with n_obs × n_vars = 961 × 630
    obs: 'n_genes'
    var: 'gene_ids', 'feature_types', 'mito', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg'
<Compressed Sparse Column sparse matrix of dtype 'float32'
	with 541595 stored elements and shape (961, 630)>
  Coords	Values
  (0, 0)	1.663979172706604
  (1, 0)	1.2755212783813477
  (2, 0)	1.2161691188812256
  (3, 0)	1.5892910957336426
  (4, 0)	0.929469883441925
  (5, 0)	1.5196822881698608
  (6, 0)	1.435987949371338
  (7, 0)	0.571940541267395
  (8, 0)	1.2916241884231567
  (9, 0)	1.1696652173995972
  (10, 0)	1.3135960102081299
  (11, 0)	1.957932949066162
  (12, 0)	1.6731693744659424
  (13, 0)	0.8780140280723572
  (14, 0)	1.9093077182769775
  (15, 0)	1.2658189535140991
  (16, 0)	0.7162562608718872
  (17, 0)	2.0933988094329834
  (18, 0)	1.5407360792160034
  (19, 0)	0.6370176672935486
  (20, 0)	0.6187309622764587
  (21, 0)	0.7288784384727478
  (22, 0)	1.0599384307861328
  (23, 0)	1.5

,n_genes
AAACACCAATAACTGC-1,2845
AAACCGGAAATGTTAA-1,2091
AAACCGGGTAGGTACC-1,2301
AAACCGTTCGTCCAGG-1,2324
AAACTGCTGGCTCCAA-1,3262
...,...
TTGGTCACACTCGTAA-1,3413
TTGGTTGCGGTGCGCG-1,2244
TTGTAATCCGTACTCG-1,2846
TTGTTCAGTGTGCTAC-1,2413


,gene_ids,feature_types,mito,n_cells,highly_variable,means,dispersions,dispersions_norm
ENO1,ENSG00000074800,Gene Expression,False,898,False,1.402186,0.306942,-0.378558
PARK7,ENSG00000116288,Gene Expression,False,783,False,1.034365,0.163946,-0.647850
CLIC4,ENSG00000169504,Gene Expression,False,884,False,1.405928,0.313264,-0.362592
LAPTM5,ENSG00000162511,Gene Expression,False,810,True,1.289964,0.972501,2.399827
HMGN2,ENSG00000198830,Gene Expression,False,914,False,1.399874,0.207616,-0.629367
...,...,...,...,...,...,...,...,...
MT-ND3,ENSG00000198840,Gene Expression,True,961,True,2.919120,1.196903,0.502178
MT-ND4L,ENSG00000212907,Gene Expression,True,781,True,1.008474,0.219138,-0.025730
MT-ND4,ENSG00000198886,Gene Expression,True,961,True,3.104381,1.125824,0.401093
MT-ND5,ENSG00000198786,Gene Expression,True,802,True,1.097405,0.245785,-0.235416


In [168]:
file_name = "filtereddata";

adata.write_csvs(data_path+file_name, skip_data=False, sep=',')

In [170]:
obs = pd.read_csv(data_path+file_name+"/obs.csv",index_col=0);
display(obs);
var = pd.read_csv(data_path+file_name+"/var.csv");
display(var);

dataset = pd.read_csv(data_path+file_name+"/X.csv",names=var['gene_ids']);
dataset.index = obs.index;
display(dataset);

highly_variable_genes = var[var['highly_variable']]['gene_ids'].values;
dataset = dataset[highly_variable_genes];
display(dataset);

,n_genes
AAACACCAATAACTGC-1,2845
AAACCGGAAATGTTAA-1,2091
AAACCGGGTAGGTACC-1,2301
AAACCGTTCGTCCAGG-1,2324
AAACTGCTGGCTCCAA-1,3262
...,...
TTGGTCACACTCGTAA-1,3413
TTGGTTGCGGTGCGCG-1,2244
TTGTAATCCGTACTCG-1,2846
TTGTTCAGTGTGCTAC-1,2413


,Unnamed: 0,gene_ids,feature_types,mito,n_cells,highly_variable,means,dispersions,dispersions_norm
0,ENO1,ENSG00000074800,Gene Expression,False,898,False,1.402186,0.306942,-0.378558
1,PARK7,ENSG00000116288,Gene Expression,False,783,False,1.034365,0.163946,-0.647850
2,CLIC4,ENSG00000169504,Gene Expression,False,884,False,1.405928,0.313264,-0.362592
3,LAPTM5,ENSG00000162511,Gene Expression,False,810,True,1.289964,0.972501,2.399827
4,HMGN2,ENSG00000198830,Gene Expression,False,914,False,1.399874,0.207616,-0.629367
...,...,...,...,...,...,...,...,...,...
625,MT-ND3,ENSG00000198840,Gene Expression,True,961,True,2.919120,1.196903,0.502178
626,MT-ND4L,ENSG00000212907,Gene Expression,True,781,True,1.008474,0.219138,-0.025730
627,MT-ND4,ENSG00000198886,Gene Expression,True,961,True,3.104381,1.125824,0.401093
628,MT-ND5,ENSG00000198786,Gene Expression,True,802,True,1.097405,0.245785,-0.235416


,ENSG00000074800,ENSG00000116288,ENSG00000169504,ENSG00000162511,ENSG00000198830,ENSG00000116251,ENSG00000077549,ENSG00000117519,ENSG00000159377,ENSG00000116473,...,ENSG00000198763,ENSG00000198804,ENSG00000198712,ENSG00000198899,ENSG00000198938,ENSG00000198840,ENSG00000212907,ENSG00000198886,ENSG00000198786,ENSG00000198727
AAACACCAATAACTGC-1,1.663979,0.000000,0.997728,1.814228,0.618454,2.257646,0.997728,1.814228,1.272052,0.000000,...,2.422381,2.495577,2.687636,2.060316,2.422381,2.563778,0.997728,2.563778,0.997728,2.163841
AAACCGGAAATGTTAA-1,1.275521,0.000000,1.275521,0.828676,2.008404,2.305779,0.000000,0.000000,0.000000,0.828676,...,2.305779,2.534692,2.632093,1.583269,2.534692,2.168105,0.828676,2.632093,0.000000,2.168105
AAACCGGGTAGGTACC-1,1.216169,1.216169,0.782585,1.216169,0.782585,1.936666,1.936666,0.000000,0.782585,0.782585,...,1.517620,2.869015,3.255381,2.554991,3.107637,3.384080,1.517620,3.300146,1.748934,2.643213
AAACCGTTCGTCCAGG-1,1.589291,1.589291,0.832949,0.000000,2.014964,2.014964,0.832949,0.832949,1.280984,0.000000,...,2.433677,2.541667,2.954977,2.174814,3.342930,2.954977,0.000000,2.809469,0.832949,2.727918
AAACTGCTGGCTCCAA-1,0.929470,0.569047,0.929470,0.929470,1.575450,1.964685,1.402741,0.929470,0.929470,1.193846,...,2.694526,2.838976,3.353301,3.209394,3.145484,3.298198,1.402741,3.145484,0.000000,2.694526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGGTCACACTCGTAA-1,0.562209,1.390828,1.390828,0.562209,2.052956,1.390828,0.919919,0.919919,1.182841,1.390828,...,2.052956,2.308023,3.194208,2.447863,3.062114,2.447863,0.919919,3.130341,0.562209,2.679773
TTGGTTGCGGTGCGCG-1,1.275304,0.828506,1.275304,1.275304,2.008143,1.583029,1.275304,0.000000,0.000000,0.000000,...,2.426496,2.534415,3.074466,1.275304,2.426496,2.426496,0.828506,2.720561,1.583029,1.818008
TTGTAATCCGTACTCG-1,1.908490,1.454340,1.454340,0.971093,0.971093,2.383702,0.971093,0.000000,0.599064,1.241628,...,2.126293,2.648062,2.648062,3.142046,2.704514,3.176870,1.778756,2.902998,1.778756,2.648062
TTGTTCAGTGTGCTAC-1,1.250845,1.250845,1.789522,1.556025,0.809439,1.789522,1.556025,0.000000,0.809439,0.809439,...,2.600203,2.688740,3.206248,2.600203,2.770071,3.206248,1.250845,3.302466,1.250845,2.915230


,ENSG00000162511,ENSG00000117519,ENSG00000159377,ENSG00000116473,ENSG00000163191,ENSG00000000971,ENSG00000265972,ENSG00000159176,ENSG00000143248,ENSG00000077157,...,ENSG00000198763,ENSG00000198804,ENSG00000198712,ENSG00000198899,ENSG00000198938,ENSG00000198840,ENSG00000212907,ENSG00000198886,ENSG00000198786,ENSG00000198727
AAACACCAATAACTGC-1,1.814228,1.814228,1.272052,0.000000,1.944823,1.487095,1.663979,3.146626,1.814228,0.618454,...,2.422381,2.495577,2.687636,2.060316,2.422381,2.563778,0.997728,2.563778,0.997728,2.163841
AAACCGGAAATGTTAA-1,0.828676,0.000000,0.000000,0.828676,0.000000,1.275521,0.828676,3.380527,2.534692,1.275521,...,2.305779,2.534692,2.632093,1.583269,2.534692,2.168105,0.828676,2.632093,0.000000,2.168105
AAACCGGGTAGGTACC-1,1.216169,0.000000,0.782585,0.782585,1.748934,1.748934,1.748934,2.934217,2.869015,1.748934,...,1.517620,2.869015,3.255381,2.554991,3.107637,3.384080,1.517620,3.300146,1.748934,2.643213
AAACCGTTCGTCCAGG-1,0.000000,0.832949,1.280984,0.000000,1.280984,0.000000,0.832949,3.549685,2.884867,0.000000,...,2.433677,2.541667,2.954977,2.174814,3.342930,2.954977,0.000000,2.809469,0.832949,2.727918
AAACTGCTGGCTCCAA-1,0.929470,0.929470,0.929470,1.193846,1.193846,1.193846,1.402741,3.209394,2.159388,0.000000,...,2.694526,2.838976,3.353301,3.209394,3.145484,3.298198,1.402741,3.145484,0.000000,2.694526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGGTCACACTCGTAA-1,0.562209,0.919919,1.182841,1.390828,1.562916,1.562916,0.919919,3.509914,2.950162,1.390828,...,2.052956,2.308023,3.194208,2.447863,3.062114,2.447863,0.919919,3.130341,0.562209,2.679773
TTGGTTGCGGTGCGCG-1,1.275304,0.000000,0.000000,0.000000,1.583029,1.275304,0.000000,2.802071,1.818008,0.000000,...,2.426496,2.534415,3.074466,1.275304,2.426496,2.426496,0.828506,2.720561,1.583029,1.818008
TTGTAATCCGTACTCG-1,0.971093,0.000000,0.599064,1.241628,1.241628,2.023310,0.971093,3.142046,3.029648,0.599064,...,2.126293,2.648062,2.648062,3.142046,2.704514,3.176870,1.778756,2.902998,1.778756,2.648062
TTGTTCAGTGTGCTAC-1,1.556025,0.000000,0.809439,0.809439,1.556025,1.556025,1.250845,2.980604,2.845282,0.809439,...,2.600203,2.688740,3.206248,2.600203,2.770071,3.206248,1.250845,3.302466,1.250845,2.915230


In [172]:
gene_list = dataset.columns.tolist();

mg = mygene.MyGeneInfo();

geneSyms = mg.querymany(gene_list, scopes='ensembl.gene', fields='symbol', species='human');

gene_info_dict = {};
for ele in geneSyms:
    # print(ele);
    query = ele['query'];
    if 'symbol' in ele:
        symbol = ele['symbol'];
        gene_info_dict[query] = symbol;
    
dataset.rename(columns=gene_info_dict, inplace=True);
dataset
dataset.to_csv(data_path+file_name+"/processed_data.csv",index=None);

1 input query terms found no hit:	['ENSG00000112096']
